In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


import os
import glob
import pandas as pd

# 2. Define the folder path containing your price CSV files.
# Adjust this path to where your price CSV files are stored in your Drive.
price_folder = "/content/drive/My Drive/capstone_dataset/price"

# Get list of all CSV files in the price folder.
price_files = glob.glob(os.path.join(price_folder, "*.csv"))
print("Found price files:", price_files)

# 3. Define conversion factors.
# For crops priced per bushel (BU):
weight_per_bu = {
    'BARLEY': 21.77,
    'CORN': 25.40,
    'OATS': 17.36,
    'SORGHUM': 25.49,
    'SOYBEANS': 27.22
}
# For commodities priced per pound (LB):
weight_per_lb = 0.4536  # common for BEANS, COTTON, PEANUTS, RICE

# 4. Map filenames to their commodity key.
# Make sure these filenames match exactly your CSV filenames.
commodity_map = {
    "barley_price.csv": "BARLEY",
    "beans_price.csv": "BEANS",
    "corn_price.csv": "CORN",
    "cotton_price.csv": "COTTON",
    "oats_price.csv": "OATS",
    "Peanuts_price.csv": "PEANUTS",
    "rice_price.csv": "RICE",
    "sorghum_price.csv": "SORGHUM",
    "soybeans_price.csv": "SOYBEANS"
}

# 5. Function to process each national-level price file.
def process_national_price(file_path, commodity_key):
    """
    Reads a national price CSV, converts the price to $/kg, and returns a DataFrame
    with columns: Year, Commodity, price_usd_per_kg.
    """
    df = pd.read_csv(file_path)

    # Standardize the Commodity column to uppercase.
    df['Commodity'] = df['Commodity'].str.upper()

    # Replace long commodity names with a simplified version if needed.
    replacements = {
        'BEANS, DRY EDIBLE, INCL CHICKPEAS': 'BEANS',
        'CORN, GRAIN': 'CORN',
        'SORGHUM, GRAIN': 'SORGHUM'
    }
    df['Commodity'] = df['Commodity'].replace(replacements, regex=True)

    # Initialize the new column.
    df['price_usd_per_kg'] = None

    # Determine conversion based on the "Data Item" column.
    for idx, row in df.iterrows():
        data_item = str(row['Data Item']).upper() if 'Data Item' in row else ""
        price_value = row['Value']

        if "BU" in data_item:
            # Convert from $/BU to $/kg using the weight_per_bu factor.
            conv_factor = weight_per_bu.get(commodity_key, None)
            if conv_factor is not None:
                df.at[idx, 'price_usd_per_kg'] = price_value / conv_factor
            else:
                df.at[idx, 'price_usd_per_kg'] = None
        elif "LB" in data_item:
            # Convert from $/LB to $/kg.
            df.at[idx, 'price_usd_per_kg'] = price_value / weight_per_lb
        else:
            df.at[idx, 'price_usd_per_kg'] = float('nan')

    # Keep only essential columns.
    df = df[['Year', 'Commodity', 'price_usd_per_kg']]
    return df

# 6. Process all price files and merge them.
all_price_dfs = []
for file_path in price_files:
    fname = os.path.basename(file_path)
    if fname in commodity_map:
        commodity_key = commodity_map[fname]
        processed_df = process_national_price(file_path, commodity_key)
        all_price_dfs.append(processed_df)
    else:
        print(f"WARNING: {fname} not found in commodity_map; skipping.")

# Concatenate all processed price DataFrames.
all_prices = pd.concat(all_price_dfs, ignore_index=True)
print("All national prices shape:", all_prices.shape)
print(all_prices.head(5))

# 7. Optionally, save the merged national price data to Drive.
output_price_path = "/content/drive/My Drive/capstone_dataset/merged_national_prices.csv"
all_prices.to_csv(output_price_path, index=False)
print("Merged national price data saved to:", output_price_path)

Found price files: ['/content/drive/My Drive/capstone_dataset/price/barley_price.csv', '/content/drive/My Drive/capstone_dataset/price/beans_price.csv', '/content/drive/My Drive/capstone_dataset/price/corn_price.csv', '/content/drive/My Drive/capstone_dataset/price/cotton_price.csv', '/content/drive/My Drive/capstone_dataset/price/oats_price.csv', '/content/drive/My Drive/capstone_dataset/price/Peanuts_price.csv', '/content/drive/My Drive/capstone_dataset/price/rice_price.csv', '/content/drive/My Drive/capstone_dataset/price/sorghum_price.csv', '/content/drive/My Drive/capstone_dataset/price/soybeans_price.csv']
All national prices shape: (158, 3)
   Year Commodity price_usd_per_kg
0  2023    BARLEY         0.339458
1  2023    BARLEY         0.337621
2  2022    BARLEY         0.339917
3  2022    BARLEY         0.303169
4  2021    BARLEY         0.243914
Merged national price data saved to: /content/drive/My Drive/capstone_dataset/merged_national_prices.csv


In [ ]:
# Load your merged county-level yield data (which you processed earlier).
yield_path = "/content/drive/My Drive/capstone_dataset/merged_crop_yields_kg_per_acre.csv"
all_yields = pd.read_csv(yield_path)

# Standardize key columns for merging.
all_yields['Year'] = all_yields['Year'].astype(int)
all_yields['Commodity'] = all_yields['Commodity'].str.upper()

all_prices['Year'] = all_prices['Year'].astype(int)
all_prices['Commodity'] = all_prices['Commodity'].str.upper()

# Merge on Year and Commodity.
merged_df = pd.merge(
    all_yields,
    all_prices,
    on=['Year', 'Commodity'],
    how='left'  # Use 'left' so all yield records are preserved.
)

print("Merged yield + national price data shape:", merged_df.shape)
print(merged_df.head())

# Optionally, save the merged dataset.
output_merged_path = "/content/drive/My Drive/capstone_dataset/merged_yield_national_price.csv"
merged_df.to_csv(output_merged_path, index=False)
print("Final merged dataset saved to:", output_merged_path)

Merged yield + national price data shape: (61334, 23)
  Program  Year Period  Week Ending Geo Level    State  State ANSI  \
0  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
1  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
2  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
3  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   
4  SURVEY  2023   YEAR          NaN    COUNTY  ALABAMA           1   

  Ag District  Ag District Code          County  ...  watershed_code  \
0         NaN                99  OTHER COUNTIES  ...               0   
1  BLACK BELT                40          DALLAS  ...               0   
2  BLACK BELT                40          ELMORE  ...               0   
3  BLACK BELT                40           MACON  ...               0   
4  BLACK BELT                40         MARENGO  ...               0   

   Watershed  Commodity                                   Data Item  Domain  \
0        NaN 

In [ ]:
all_yields['Year'] = all_yields['Year'].astype(int)
all_yields['Commodity'] = all_yields['Commodity'].str.upper().str.strip()

all_prices['Year'] = all_prices['Year'].astype(int)
all_prices['Commodity'] = all_prices['Commodity'].str.upper().str.strip()

In [ ]:
print(merged_df.isna().sum())

Program                  0
Year                     0
Period                   0
Week Ending          61334
Geo Level                0
State                    0
State ANSI               0
Ag District            569
Ag District Code         0
County                   0
County ANSI           7110
Zip Code             61334
Region               61334
watershed_code           0
Watershed            61334
Commodity                0
Data Item                0
Domain                   0
Domain Category          0
Value                 6899
CV (%)               45805
yield_kg_per_acre    10332
price_usd_per_kg      4778
dtype: int64


In [ ]:
print("Unique yield commodities:", all_yields["Commodity"].unique())
print("Unique price commodities:", all_prices["Commodity"].unique())

Unique yield commodities: ['CORN' 'COTTON' 'BARLEY' 'BEANS' 'OATS' 'PEANUTS' 'RICE' 'SORGHUM'
 'SOYBEANS']
Unique price commodities: ['BARLEY' 'BEANS' 'CORN' 'COTTON' 'OATS' 'PEANUTS' 'RICE' 'SORGHUM'
 'SOYBEANS']


In [ ]:
all_yields['Year'] = all_yields['Year'].astype(int)
all_prices['Year'] = all_prices['Year'].astype(int)

all_yields['Commodity'] = all_yields['Commodity'].str.upper().str.strip()
all_prices['Commodity'] = all_prices['Commodity'].str.upper().str.strip()

In [ ]:
replacements = {
    'CORN, GRAIN': 'CORN',
    'SORGHUM, GRAIN': 'SORGHUM',
    # etc.
}
all_yields['Commodity'] = all_yields['Commodity'].replace(replacements, regex=True)

In [ ]:
import pandas as pd

# ----- Step 1: Load and Prepare Land Value Data -----
lv_file = "/content/drive/My Drive/capstone_dataset/land_value_data.csv"
lv_df = pd.read_csv(lv_file)

# Rename the column holding land value data to "Land_Value"
if 'Value' in lv_df.columns:
    lv_df.rename(columns={'Value': 'Land_Value'}, inplace=True)
elif 'value' in lv_df.columns:
    lv_df.rename(columns={'value': 'Land_Value'}, inplace=True)
elif 'VALUE' in lv_df.columns:
    lv_df.rename(columns={'VALUE': 'Land_Value'}, inplace=True)
else:
    raise KeyError("No column for land value found in the dataset.")

print("Land Value columns after renaming:", lv_df.columns.tolist())

# Ensure 'Year' is an integer and standardize 'State' and 'County'
lv_df['Year'] = lv_df['Year'].astype(int)
lv_df['State'] = lv_df['State'].str.upper().str.strip()
if 'County' in lv_df.columns:
    lv_df['County'] = lv_df['County'].str.upper().str.strip()
else:
    raise KeyError("Column 'County' not found in land value dataset.")

# ----- Step 2: Load Final Clean Dataset -----
final_file = "/content/drive/My Drive/capstone_dataset/final_clean_dataset.csv"
final_df = pd.read_csv(final_file)

# Standardize key columns in the final clean dataset
final_df['Year'] = final_df['Year'].astype(int)
final_df['State'] = final_df['State'].str.upper().str.strip()
if 'County' in final_df.columns:
    final_df['County'] = final_df['County'].str.upper().str.strip()
else:
    raise KeyError("Column 'County' not found in final clean dataset.")

# ----- Step 3: Merge the Datasets on (Year, State, County) -----
# We keep all rows from final_df and add Land_Value from the land value dataset.
merged_df = pd.merge(final_df,
                     lv_df[['Year', 'State', 'County', 'Land_Value']],
                     on=['Year', 'State', 'County'],
                     how='left')

print("Shape of the merged dataset:", merged_df.shape)
print("Sample merged data (selected columns):")
print(merged_df[['Year', 'State', 'County', 'Land_Value']].head())

# ----- Step 4: Save the Final Merged Dataset -----
output_path = "/content/drive/My Drive/capstone_dataset/final_dataset_with_land_value.csv"
merged_df.to_csv(output_path, index=False)
print("Final merged dataset saved to:", output_path)

Land Value columns after renaming: ['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed', 'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Land_Value', 'CV (%)']
Shape of the merged dataset: (48039, 24)
Sample merged data (selected columns):
   Year    State          County Land_Value
0  2023  ALABAMA  OTHER COUNTIES        NaN
1  2023  ALABAMA          DALLAS        NaN
2  2023  ALABAMA          ELMORE        NaN
3  2023  ALABAMA           MACON        NaN
4  2023  ALABAMA         MARENGO        NaN
Final merged dataset saved to: /content/drive/My Drive/capstone_dataset/final_dataset_with_land_value.csv


In [ ]:
import pandas as pd
import numpy as np

# ----- Step 1: Load and Prepare Land Value Data -----
lv_file = "/content/drive/My Drive/capstone_dataset/land_value_data.csv"
lv = pd.read_csv(lv_file)

# Print original columns for inspection
print("Original columns:", lv.columns.tolist())

# Rename the column holding land value data to "Land_Value"
if 'Value' in lv.columns:
    lv.rename(columns={'Value': 'Land_Value'}, inplace=True)
elif 'value' in lv.columns:
    lv.rename(columns={'value': 'Land_Value'}, inplace=True)
elif 'VALUE' in lv.columns:
    lv.rename(columns={'VALUE': 'Land_Value'}, inplace=True)
else:
    raise KeyError("No column for land value found in the dataset.")

print("Updated columns:", lv.columns.tolist())

# Ensure 'Year' is integer
lv['Year'] = lv['Year'].astype(int)

# Standardize 'State' and 'County'
lv['State'] = lv['State'].str.upper().str.strip()
if 'County' in lv.columns:
    lv['County'] = lv['County'].str.upper().str.strip()
else:
    raise KeyError("Column 'County' not found in land value dataset.")

# Remove commas from Land_Value and convert to numeric
lv['Land_Value'] = lv['Land_Value'].astype(str).str.replace(',', '', regex=False)
lv['Land_Value'] = pd.to_numeric(lv['Land_Value'], errors='coerce')

# ----- Step 2: Interpolate Annual Land Value Data -----
# We want a Land_Value for every year from 2010 to 2025.
full_years = np.arange(2010, 2026)  # 2010 to 2025 inclusive

def interp_group(group):
    # Set 'Year' as index and sort
    group = group.set_index('Year').sort_index()
    # Reindex to include every year in the full range
    group = group.reindex(full_years)
    # Interpolate missing Land_Value using linear interpolation
    group['Land_Value'] = group['Land_Value'].interpolate(method='linear')
    # Fill any NaN at the boundaries with forward/backward fill
    group['Land_Value'] = group['Land_Value'].fillna(method='ffill').fillna(method='bfill')
    return group.reset_index()

# Group by State and County
lv_interp = lv.groupby(['State', 'County']).apply(interp_group).reset_index(drop=True)
# Keep only the columns needed for merging
lv_final = lv_interp[['Year', 'State', 'County', 'Land_Value']]
print("Interpolated Land Value Data (first 10 rows):")
print(lv_final.head(10))

# ----- Step 3: Load Final Clean Dataset and Merge Land Value Data -----
final_file = "/content/drive/My Drive/capstone_dataset/final_clean_dataset.csv"
final_df = pd.read_csv(final_file)

# Standardize key columns in final_df
final_df['Year'] = final_df['Year'].astype(int)
final_df['State'] = final_df['State'].str.upper().str.strip()
final_df['County'] = final_df['County'].str.upper().str.strip()

# Merge on Year, State, and County (left join to keep all rows from final_df)
merged_df = pd.merge(final_df, lv_final, on=['Year', 'State', 'County'], how='left')

print("Shape of the merged dataset:", merged_df.shape)
print("Sample merged data (selected columns):")
print(merged_df[['Year', 'State', 'County', 'Land_Value']].head(10))

# ----- Step 4: Save Final Merged Dataset -----
final_output_path = "/content/drive/My Drive/capstone_dataset/final_dataset_with_land_value.csv"
merged_df.to_csv(final_output_path, index=False)
print("Final merged dataset with land value saved to:", final_output_path)

Original columns: ['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed', 'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value', 'CV (%)']
Updated columns: ['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed', 'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Land_Value', 'CV (%)']


Streaming output truncated to the last 5000 lines.
<ipython-input-26-e0ace159d7e1>:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['Land_Value'] = group['Land_Value'].fillna(method='ffill').fillna(method='bfill')
<ipython-input-26-e0ace159d7e1>:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['Land_Value'] = group['Land_Value'].fillna(method='ffill').fillna(method='bfill')
<ipython-input-26-e0ace159d7e1>:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['Land_Value'] = group['Land_Value'].fillna(method='ffill').fillna(method='bfill')
<ipython-input-26-e0ace159d7e1>:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  g

Interpolated Land Value Data (first 10 rows):
   Year    State   County  Land_Value
0  2010      NaN      NaN      2285.0
1  2011      NaN      NaN      2285.0
2  2012  ALABAMA  AUTAUGA      2285.0
3  2013      NaN      NaN      2269.0
4  2014      NaN      NaN      2253.0
5  2015      NaN      NaN      2237.0
6  2016      NaN      NaN      2221.0
7  2017  ALABAMA  AUTAUGA      2205.0
8  2018      NaN      NaN      2291.2
9  2019      NaN      NaN      2377.4
Shape of the merged dataset: (48039, 24)
Sample merged data (selected columns):
   Year    State          County  Land_Value
0  2023  ALABAMA  OTHER COUNTIES         NaN
1  2023  ALABAMA          DALLAS         NaN
2  2023  ALABAMA          ELMORE         NaN
3  2023  ALABAMA           MACON         NaN
4  2023  ALABAMA         MARENGO         NaN
5  2023  ALABAMA           PERRY         NaN
6  2023  ALABAMA         BALDWIN         NaN
7  2023  ALABAMA        ESCAMBIA         NaN
8  2023  ALABAMA          MONROE         NaN
9  202

In [ ]:
import pandas as pd
import numpy as np

# ---------------------------
# Step 1: Load and Standardize the Final Merged Dataset
# ---------------------------
final_file = "/content/drive/My Drive/capstone_dataset/final_dataset_with_land_value.csv"
df = pd.read_csv(final_file)

# Standardize key columns
df['Year'] = df['Year'].astype(int)
df['State'] = df['State'].str.upper().str.strip()
df['County'] = df['County'].str.upper().str.strip()

# ---------------------------
# Step 2: Define Function to Fill Missing Land_Value per Group
# ---------------------------
def fill_land_value_for_group(group):
    """
    For a given (State, County) group, use available Land_Value data (if >=2 points exist)
    to fit a linear model (Land_Value = slope * Year + intercept) and update the Land_Value column.
    If only one value is available, use that value for all rows.
    """
    # Get observed values (non-missing) aggregated by Year
    observed = group.dropna(subset=['Land_Value'])
    observed_yearly = observed.groupby('Year', as_index=False)['Land_Value'].mean()

    if len(observed_yearly) >= 2:
        # Fit linear model using np.polyfit
        slope, intercept = np.polyfit(observed_yearly['Year'], observed_yearly['Land_Value'], 1)
        # Predict Land_Value for every row in the group using the fitted model
        group['Land_Value'] = group['Year'].apply(lambda yr: slope * yr + intercept)
    elif len(observed_yearly) == 1:
        # Only one observed value; fill all rows with that constant value
        constant_val = observed_yearly['Land_Value'].iloc[0]
        group['Land_Value'] = constant_val
    # If no observed value, the Land_Value remains missing
    return group

# ---------------------------
# Step 3: Apply the Function Groupwise to Update Land_Value In Place
# ---------------------------
# Group by State and County and update the Land_Value column using our function.
df_updated = df.groupby(['State', 'County'], group_keys=False).apply(fill_land_value_for_group)

# ---------------------------
# Step 4: Save the Updated Dataset
# ---------------------------
output_path = "/content/drive/My Drive/capstone_dataset/final_dataset_with_land_value_filled.csv"
df_updated.to_csv(output_path, index=False)
print("Updated dataset saved to:", output_path)

<ipython-input-27-395cf181a55a>:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_updated = df.groupby(['State', 'County'], group_keys=False).apply(fill_land_value_for_group)


Updated dataset saved to: /content/drive/My Drive/capstone_dataset/final_dataset_with_land_value_filled.csv
